In [1]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pylab import plot,show,hist
from scipy.stats.kde import gaussian_kde
from scipy.stats import norm, chi2_contingency
import statsmodels.api as sm
from numpy import linspace,hstack
from pylab import plot,show,hist
import pydot
#%config InlineBackend.figure_format = 'svg' для большей четкости графиков
matplotlib.style.use('ggplot')
%matplotlib inline

#Стандартизация данных
from sklearn import preprocessing

#Для построения диаграмм рассеивания
from pandas.plotting import scatter_matrix

#Графика для интерпретации моделей
from IPython.display import Image
from sklearn.tree import export_graphviz
from subprocess import call

#Иерархический кластерный анализ
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

#Кластерный анализ методом К-средних
from sklearn.cluster import KMeans

#Расщепление на обучающую и тестовую выборки
from sklearn.model_selection import train_test_split

#Деревья решений для задачи классификации
from sklearn.tree import DecisionTreeClassifier

#Деревья решений для задач регрессии 
from sklearn.ensemble import RandomForestClassifier

#Калибровка деревьев решений
from sklearn.calibration import CalibratedClassifierCV


os.chdir(r'C:\Users\Mr Alex\Documents\GitHub\FlightPreparence')
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
           'marital-status', 'occupation', 'relationship', 'race', 'sex',
           'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']


#df = pd.read_csv('AmesHousing.txt', sep="\t", header = 0, index_col=False)
#df = pd.read_csv('town_1959_2.csv', header = 0,)
#df = pd.read_csv('swiss_bank_notes.csv', index_col=0)
#df = pd.read_csv('beverage_r.csv', sep=";", index_col='numb.obs')
#df = pd.read_csv('Protein Consumption in Europe.csv', sep=';', decimal=',', index_col='Country')
#df = pd.read_csv('assess.dat', sep='\t', index_col='NAME')
#df = pd.read_csv('Albuquerque Home Prices_data.txt', sep='\t')
#df = pd.read_csv('agedeath.dat.txt', sep='\s+', header=None, names=['group', 'age', 'index'])
#df = pd.read_csv('interference.csv')
#df = pd.read_csv('diamond.dat', header=None, sep='\s+', names=['weight', 'price'])
#df = pd.read_csv('Credit.csv', sep=';', encoding='cp1251')
#df = pd.read_csv('adult.test', header=None, names=columns, na_values=' ?', skiprows=1)
#df = pd.read_csv('Wine.txt', sep='\t', header=0)
#df = pd.read_csv('monthly-car-sales-in-quebec-1960.csv', sep=';', header=0, parse_dates=[0])
#df = pd.read_csv('stickleback.csv', sep=';', decimal=',')
df = pd.read_csv('Swiss Fertility.csv', sep=';', decimal=',', index_col=0)

In [2]:
def two_histograms(x, y):
    """
    Функция, которая построит две гистограммы на одной картинке.
    Дополнительно пунктирными линиями указываются средние значения выборок.
    x: вектор pd.Series,
    y: вектор pd.Series
    """
    x.hist(alpha=0.5, weights=[1./len(x)]*len(x))
    y.hist(alpha=0.5, weights=[1./len(y)]*len(y))
    plt.axvline(x.mean(), color='red', alpha=0.8, linestyle='dashed')
    plt.axvline(y.mean(), color='blue', alpha=0.8, linestyle='dashed')
    plt.legend([x.name, y.name])
    
def regression_coef(model, X, y):
    """
    Функция для определения статистической значимости регрессионных коэффициентов
    """
    coef = pd.DataFrame(zip(['intercept'] + X.columns.tolist(), [model.intercept_] + model.coef_.tolist()),
                    columns=['predictor', 'coef'])
    X1 = np.append(np.ones((len(X),1)), X, axis=1)
    b = np.append(model.intercept_, model.coef_)
    MSE = np.sum((model.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
    var_b = MSE * (np.linalg.inv(np.dot(X1.T, X1)).diagonal())
    sd_b = np.sqrt(var_b)
    t = b / sd_b
    coef['pvalue'] = [2 * (1 - stats.t.cdf(np.abs(i), (len(X1) - 1))) for i in t]
    return coef


def scale_features(df):
    """
    Функция для стандартизации переменных в датафрейме
    """
    scaled = preprocessing.StandardScaler().fit_transform(df)
    scaled = pd.DataFrame(scaled, columns=df.columns)
    return scaled

In [ ]:
#Классификация - приписание объекта к классу на основании ключевой (группирующей) переменной или совокупности его характеристик
#Типы переменных. Количественные(непрерывные, дискретные). Номинальные (несравниваемые). Ранговые (порядковые)
#Гистограмма частот - форма распределения количественного признака
#Описательная статистика. Меры центральной тенденции. Меры изменчивости (Размах - Xmax-Xmin)
#МЦТ. Мода - самый частый признак. Медиана - делит упорядоченное множество пополам. Среднее (Математическое ожидание, EX)
#Дисперсия D - средний квадрат отклонений индивидуальных значений от средней величины. С ростом n, дисперсия сокращается
#D = сумма(Xинд - Xсред)**2/n-1. Хсред генеральной совокупности обозначается как мю, М
#Стандартное отклонение, "сигма", sd = D**0.5

In [ ]:
#Иерархический кластерный анализ разделяет объекты на группы (стратификация). Число групп заранее неизвестно
#Кластерный анализ позволяет сократить число наблюдений и проинтерпретировать их
#Схожесть внутри кластера отображается как расстоянием между близкими объектами на диаграмме кластеров
#Расстояние можно рассчитать методами: Евклида(или квадрата Евклида), Блок(Манхеттен), Хэмминга(для слов) и тд.
#Манхеттен предпочтительнее, когда нет больших различий в рандомных переменных, потому что вес аномалий тогда меньше
#Расстояние между кластерами рассчитывается:
#Метод Варда (WARD) - позволяет работать с шаровыми скоплениями
#Метод ближайших соседей (позволяет определять ленточные кластеры)
#Средневзвешенное расстояние: среднее для суммы всех расстояний (также для ленточных)
#Центроид: расстояние между кластерами равно расстоянию между их центрами тяжести
#Методы дальнего и ближайшего соседа: расстояние между самыми дальними\близкими объектами есть межкластер
#Метод расстояния Sorencen-Dice Q = 2*|A^B|/|A|+|B|. Не работает если множества слабо пересекаются

In [ ]:
#Алгоритмы кластерного анализа. Объекты группируются с ближайшими, пока нет скачка в расстояниях для следующего слияния
#Момент для прекращения слияния определяется дендрограммой (для умеренного числа объектов)
#Каменистая осыпь/локоть показывают скачок (резкий взлет графика) шагов объединений, когда кластеризуются тысячи объектов
#Задача аналитика: отобрать переменные, выбрать метод стандартизации, установить расстояние между кластерами и между объектами 

In [ ]:
#Объект, в котором будет хранится информация о последовательном слиянии кластеров
link = linkage(df, 
               'ward', #метод межкластера 
               'euclidean' #метод межобъектов
              )

#link - матрица (n-1) x 4, где n - число наблюдений. 
#Каждая строка - результат слияния очередной пары кластеров с номерами link[i, 0] и link[i, 1]. 
#Новому кластеру присваивается номер n + i 
#link[i, 2] означает расстояние между слитыми кластерами, а link[i, 3] - размер нового кластера.
link[:5]

In [ ]:
#Построение дендрограммы
dn = dendrogram(link, orientation='right')

In [ ]:
#Ключевые характеристики кластеров
#В колонку cluster запишем номер кластера объекта с помощью функции fcluster. 
#Аргументы: linkage, пороговое значение для межкластера (либо число кластеров), criterion: distance для остановки разбиения 
# Останавливаем объединение, если расстояние между кластерами превышает 3
df['cluster'] = fcluster(link, 3, criterion='distance')
#Доля объектов в кластере, которые имеют соответствующие характеристики
df.groupby("cluster").mean()

In [ ]:
#Кластерный анализ методом К-средних
#инициализация модели
model = KMeans(n_clusters=2, random_state=42) #random_state - зерно датчика случайных чисел. Для воспроизводимости результата 
#При каждом новом вызове с одинаковыми random_state модель будет давать одинаковые результаты

In [ ]:
#подгонка модели по данным из БД
model.fit(df)

#Результат кластеризации на данных из БД
model.labels_

#координаты центров кластеров
model.cluster_centers_

#Добавление в кластер данных. Предсказание для новых наблюдений. Метод predict
new_items = [
    [1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0]
]
model.predict(new_items)

In [ ]:
#Число кластеров можно определить через график локтя для для разного числа кластеров
#Метод inertia_ вернёт сумму расстояний от каждой точки данных до центра ближайшего у ней кластера 
#Кластеризацию можно считать условно хорошей, когда инерция перестаёт сильно уменьшаться при увеличении числа кластеров
K = range(1, 11)
models = [KMeans(n_clusters=k, random_state=42).fit(df) for k in K]
dist = [model.inertia_ for model in models]

#График локтя
plt.plot(K, dist, marker='o')
plt.xlabel('k')
plt.ylabel('Sum of distances')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
# В колонке NR находится номер объекта, его нужно исключить из данных для кластеризации
del df['NR']

#Оптимизируем модель, меняя число задаваемых кластеров на основании графика локтя
model = KMeans(n_clusters=4, random_state=42)
model.fit(df)
df['cluster'] = model.labels_
df.groupby('cluster').mean()

#Смотрим к какому кластеру какие объекты относятся
df['cluster'].sort_values()

In [ ]:
#Альфа-это уровень значимости(0.05, 0.01. 0.005). Определеяет число ошибок первого рода. На второго рода влияет размер выборки
#Т- это статистика критерия. Если T<Cальфа, то верна нулевая гипотеза
#Cальфа- это критическое значение. Вероятность отвергнуть правильную гипотезу(T>C) не должна превышать А(альфа)
#p-value показывает насколько часто статистика критерия в верной гипотезе будет превышать реальные значения p=P{T>Tэксп}
#Если p<A, гипотезу отвергаем. Если p>A, гипотезу не отвергаем. Проверяются все условия, при которых критерий будет работать

In [ ]:
#Z-метка (организация выборок так, чтобы они мало отличались от нормального распределения)

s1 = 135       # успех в выборке А
n1 = 1781      # выборка А
s2 = 47        # успех в выборке Б
n2 = 1443      # выборка Б
p1 = s1/n1               #  оценка вероятности успеха выборка А
p2 = s2/n2               #  оценка вероятности успеха выборка Б
p = (s1 + s2)/(n1+n2)    #  оценка вероятности успеха выборки А+Б
z = (p2-p1)/ ((p*(1-p)*((1/n1)+(1/n2)))**0.5) #Z-метка 

p_value = norm.cdf(z) #Функция распределения нормального распределения

#  z-метка и p-значение
print(['{:.12f}'.format(a) for a in (abs(z), p_value * 2)])
#Нулевая гипотеза отвергнута, статистические доли отличаются

#То же самое, но со встроенным методом библиотеки statsmodels
z1, p_value1 = sm.stats.proportions_ztest([s1, s2], [n1, n2])
print(['{:.12f}'.format(b) for b in (z1, p_value1)])

In [ ]:
#Тест Стьюдента на независимость переменных
x = df[df['group'] == 'sovr']['age']
y = df[df['group'] == 'aris']['age']
x.name, y.name = 'sovr', 'aris'
two_histograms(x, y) #Данные условно нормальны. 

#Проверим c помощью критерия Флигнера-Килина, равны ли дисперсии.
res = stats.fligner(x, y)
print('p-value: ', res[1]) #p-value низкое, гипотезу о равенстве дисперсий отвергаем, наблюдаемые объекты несвязные 

#Гипотезу о равенстве средних значений будем проверять с помощью теста Стьюдента при неравных дисперсиях
res = stats.ttest_ind(x, y, equal_var=False) #Опция equal_var=False говорит, что равенство дисперсии не предполагать
print('p-value: ', res[1]) #P-значение значительно меньше альфы, гипотеза о равенстве отвергается

#Ищем зависимость цены от переменной COR=1 и 0. Чтобы применить Стьюдента, проверим нормальность данных и равенство дисперсий
#Заменяем -9999 (здесь=пустое) на корректное пустое значение.
df = df.replace(-9999, np.nan)
#Сохраним в отдельные переменные выборки, которые собираемся сравнивать.
x = df[df['COR'] == 1]['PRICE']
y = df[df['COR'] == 0]['PRICE']
x.name, y.name = 'corner', 'not corner'    

two_histograms(x, y)  #Видно, что выбросы не дают применить Стюдента и нужно пробовать Манна-Витни

In [ ]:
#Распределение Стьюдента (t-distribution) для n<30 - более высокие хвосты распределений.Число степеней свободы df=n-1
#t заменяет Z в распределении Стьюдента. t=(Xинд-M)/(sd/n**0.5)
#Помимо средних также нужно сравнить дисперсии D (тест Флигнера-Килина) и медианы (много n - тест Муда, мало n - Манн-Витни) 

In [ ]:
#Парный t-критерий Стьюдента.  X1сред - Х2сред = А , se=((sd1**2/n1)+(sd2**2/n2))**0.5 , df=n1+n2-2
#При t = A/se и df можно рассчитать p при котором M1-M2=0. То есть разницы между выборками почти нет
#Q-Q Plot показывает насколько выборочные значения соответствуют предсказанным(из нормального распределеня)
x = df['DiffCol']
y = df['Black']
x.name, y.name = 'DiffCol', 'Black'
two_histograms(x, y)

#Распределения условно нормальны. Поскольку в наблюдениях содержатся одни и те же люди, выборки связные (парные)
res = stats.ttest_rel(x, y) #Метод для парных выборок
print('p-value: ', res[1])
p-value: 0.0162416779538
#p-value низкий, гипотеза на уровне значимости 0.05 будет отвергнута, но на уровне 0.01 уже нет. 

In [ ]:
#U-критерий Манна-Витни переводит значения в ранговую (непараметрическую) шкалу и проверяет НЕ равенство медиан. P{X>Y}=P{X<Y}

res = stats.mannwhitneyu(x, y)
print('p-value:', res[1])
#p-value получилось достаточно большим, поэтому у нас нет оснований отвергнуть гипотезу. Разница медиан в выборках случайна.

In [ ]:
#Корреляция. Scatter-plot или диагратта рассеивания
#Сила и направление взаимосвязи определяется ковариацией. cov=Сумма((Xi-Xсред)*(Yi-Yсред))/N-1
#Коэффициент корреляции Пирсона находится в промежутке [-1; 1] и считается как Rxy=cov/SDx*SDy
#Коэффициент детерминации r**2 показывает влияние дисперсии одной переменной на другую в промежутке [0; 1]
#Коэффициент Спирмена позволяет блокировать выбросы через ранги. d=X-Y. Rs=1-6*сумма d**2/N(N**2-1)
#Часто корреляция обусловлена скрытой переменной

#Корреляция цены и размера
plt.scatter(df['PRICE'], df['SQFT'])

res = stats.pearsonr(df['PRICE'], df['SQFT']) #Допускаем что коэфффициент корреляции=0, но гипотеза отвергнута

print('Pearson rho: ', res[0])
print('p-value: ', res[1])

In [ ]:
#Стандартизация позволяет сделать вес важных переменных соизмеримым. Min=0(-1), max=1. ИЛИ Z
#Z-Стандартизация: преобразование в тип, где М=0, sd = 1. Правило одной, двух и трех "сигм"
#Z=(Xинд-М)/sd Пример: по таблице Z, где Хсред=150, sd=8, превышать Xинд будет 0.5z или 30%
#Z=(Xсред-M)/se =(18,5-20)/0.5 = -3. Вероятность получить такой результат p = 0.0027

#Если в БД нет единой метрики, то стандартизируем данные
norm = preprocessing.StandardScaler()
norm.fit(df)
X = norm.transform(df)

In [ ]:
#Дисперсионный Анализ. Если межгрупповой показатель изменчивости сильно превышает внутригрупповой, то средние разнятся
#SST - общая сумма квадратов показывает общую изменчивость данных. Сумма(Xинд-Xсред)**2  SST = SSW+SSB
#SSW - сумма квадратов внутригрупповая. Сумма(X1инд-Х1сред)**2 + ...(XNинд-ХNсред)**2
#SSB - сумма квадратов межгрупповая. SSB= n(X1сред - Хсред)**2 + ...n(XNсред-Хсред)**2

In [ ]:
#Распределение Фишера, F-значение. F=(ssb/n-1)/(ssw/N-n). При верности нулевой гипотезы значения F очень маленькие
#Поправка Бонферрони на множественную проверку гипотез. a = ai/n  НО: мешает получить значимые уровни различия
#FDR или критерий Тьюки считает p-уровень для сравниваемых пар Xтэ=Xa-Xб
#Двухакторный дисперсионный анализ SStotal=SSW+SSBa +SSBb + SSBa*SSBb
#Взаимодействие факторов в ANOVA
#Дисперсионный анализ требует нормальности распределения зависимой переменной и гомогенности дисперсии(тест Левена)

In [ ]:
#Задача распознавания наименований или порядков через деревья классификации. И чисел через регрессию
#Помимо внутренних параметров (заданных изначально), есть еще внешние (задаваемые аналитиком)
#Выбор модели с помощью обучающей/тестовой выборок через наименьшую среднюю квадратичную ошибку 
#Критерий качества Q - сумма модулей ошибок или сумма квадратов ошибок или процент ошибок и т.д.
#Валидация - метод проверки выбранной модели на ее адекватность
#Регуляризация - инструмент проверки моделей

In [ ]:
#CART - Classification and regression trees
#деление матрицы прямыми\гиперплоскостями, чтобы в ограниченных областях доминировали схожие объекты
#Узел(node) - множество, которое расщепляется. Родительский, потомок, конечный. 
#Пороговое значение - эталон для сравнения
#Ограничения задаются оператором. На кол-во слоев, на свойство потомков, на родителя, на правила остановки 
#Чистота - порядок разделения выборки на части, в каждой из которых "загрязнение" данных меньше
#Критерий загразненности(вероятность принадлежать к классу P) измеряется Энтропией, Индексом Джини или Ошибкой Классификации
#Энтропия H1 = -СуммаP*log2P. Индекс Джини H2 = 1-СуммаP**2 = СуммаP*(1-P). Ошибка Классификации H3 = 1-maxP
#Дельта H - вклад переменной в очищение. Считаем суммы для каждой и получаем информативность переменной

#Задача кредитного скоринга
# Правильный ответ записываем в вектор y
y = df[u'кредит']
# Удаляем колонку с правильным ответом
X = df.drop(u'кредит', axis=1)

from sklearn.tree import DecisionTreeClassifier

#Инициализируем и обучаем модель
model = DecisionTreeClassifier(random_state=42,
                               # функция для impurity ('gini' или 'entropy')
                               criterion='gini',
                               # максимальная глубина дерева
                               max_depth=5,
                               # минимальное число элементов в узле для разбиения (может быть долей)
                               min_samples_split=5,
                               # минимальное число элементов в листе (может быть долей)
                               min_samples_leaf=5,
                               # минимальное значение дельты impurity
                               # min_impurity_decrease=0,
                               # веса для классов (можно дополнительно штрафовать за ошибку в нужных классах).
                               # поддерживает опцию 'balanced'.
                               class_weight=None
                               
                              )

model.fit(X, y)

#Для интерпретации получившейся модели изображаем её в виде дерева предикатов (решающих правил)

export_graphviz(model,
                out_file='tree.dot',
                #задать названия фич
                #feature_names=X.columns,
                class_names=None,
                #показывать названия полей у численных значений внутри узла
                label='all',
                #раскрашивать узлы в цвет преобладающего класса
                filled=True,
                #показывать значение impurity для каждого узла
                impurity=True,
                #показывать номера узлов
                node_ids=True,
                #Показывать доли каждого класса в узлах (а не количество)
                proportion=True,
                #Повернуть дерево на 90 градусов (вертикальная ориентация)
                rotate=True,
                #Число точек после запятой для отображаемых дробей
                #precision=3
               )

#Преобразуем файл .dot в .png
#node - номер узла, X[1]<=1.5 правило расщепления, gini, samples-доля наблюдений попавших в узел, p-value (p0, pX)
(graph,) = pydot.graph_from_dot_file('tree.dot')
graph.write_png('tree.png')
Image("tree.png")

#Модель позволяет оценить ценность (importance) и эффективность каждой фичи, считая для каждой из сумму дельты H 
pd.DataFrame({'feature': X.columns,
              'importance': model.feature_importances_}).sort_values('importance', 
            ascending=False
            )

#Метод predict позволяет получить предсказания классов для входного списка элементов (подаём на вход матрицу)
# Предсказание класса для новых элементов
new_item = [1, 1, 1, 1]
model.predict([new_item])


In [ ]:
#Расщепление на обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,
                                                    # доля объёма тестового множества
                                                    test_size=0.2)
#обучаем модель
model.fit(X_train, y_train)

#Строим предсказание модели на тестовом множестве
y_pred = model.predict(X_test)

#Оценка качества классификатора: доля совпавших ответов в y_pred и y_test, или считаем точность и полноту
#Если доля в обучающем выше тестового, означает переобученность модели. Нужно упрощать модель
#Матрица ошибок  𝐶=(𝑐𝑖,𝑗) , где  𝑐𝑖,𝑗 количество элементов класса 𝑖 , которым классификатор присвоил класс 𝑗 
#Точность(precision) - доля правильно классифицированных объектов в найденных классификатором. 
#Полнота(recall) - доля этих объектов НА САМОМ ДЕЛЕ
conf_mat = metrics.confusion_matrix(y_test, y_pred)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

#Гармоническое среднее F1 = 2*точность*полнота/(точность+полнота). Считается с помощью classification_report
print(metrics.classification_report(y_pred, y_test))

In [ ]:
#Деревья решений для задач регрессии (отклик не дискретный, а непрерывный). Методы схожы с деревом классификации
#Предпочтительнии линейной регрессии, когда зависимость не линейная :)
#В этом случа Дельта H = сумма квадратов ошибок
#Prune (обрезание) - очистка от узлов, которые не нужны, через добавление третьей выборки (валидации)

#Случайный лес. Ключевые параметры:
#ntree - число деревьев(в начале макс, потом сокращать), mtry - число переменных в выборке (M**0.5)
#sampsize - число наблюдений в подвыборке(0.632*N для декорреляции), nodesize - мин. число наблюдений в узле (10) 
#replace - подвыборка с  возвращением или без
#out-of-bag - неиспользуемая при обучении часть выборки, используется в качестве предварительного теста модели

#Если несколько классов, но хочется сделать классификацию строго бинарной, то разбиваем на группы ДА и НЕТ
df['Desired1(3)'] = df['Desired1(3)'].replace(0, 1)


model = RandomForestClassifier(random_state=42, #зерно датчика случайных чисел
                               #число деревьев в лесу
                               n_estimators=30,
                               #функция для дельта H, impurity ('gini' или 'entropy')
                               criterion='gini',
                               #Макс число слоев
                               max_depth=5,
                               #Вычислять out-of-bag ошибку
                               oob_score=True,
                               #использовать результаты предыдущего вызова и нарастить предыдущий лес 
                               warm_start=False,
                               #веса классов для балансировки выборки для обучения
                               class_weight=None
                               
                              )

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(metrics.classification_report(y_pred, y_test))

print('Out-of-bag score: {0}'.format(model.oob_score_)) 

pd.DataFrame({'feature': X.columns,
              'importance': model.feature_importances_}).sort_values('importance', ascending=False)

In [ ]:
#Калибровка классификаторов. Нужна для рабочих моментов, чтобы лучше понять машину 
#Уточнение выданных машиной значений, с учетом "вероятности" попасть в класс
#Можно задать пороговые значения, которые определят класс. В промежутке между ними машина скажет "не знаю"
#Калибровка - это пересчет выходных значений, чтобы про них м.б. сказать - это Вероятность
#Изотоническая регрессия - это линия, у которой вектор не убывает
#Логистическая кривая, метод Платта - неубывающая линия, которая приблизит Pi к P


# Строим предсказание модели с возможностью посмотреть на вероятности принадлежать к классу
y_pred_train2 = model.predict_proba(X_train)
y_pred_test2 = model.predict_proba(X_test)

#Завершаем построенную машину калибровкой
model_sigmoid = CalibratedClassifierCV(model, 
                                       cv=2, 
                                       method='sigmoid' #Или "isotonic"
                                      )


#Обучаем калибровку на выборке валидации
model_sigmoid.fit(X_train, y_train)

#Смотрим на вероятности после калибровки
model_sigmoid.predict_proba(X_test)

# Предсказание класса для новых элементов
new_item = [1, 1, 1, 1]
model.predict([new_item])